# Backtester Testing Framework

This notebook demonstrates how to use the testing framework to validate our backtesting system. We'll cover:

1. Unit testing with different market scenarios
2. Testing edge cases (missing data, extreme price movements)
3. Validating backtest results
4. Setting up automated test cases

In [ ]:
# Import required libraries
import sys
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
import unittest
from pprint import pprint

# Add the project root to the path so we can import our modules
project_root = os.path.abspath(os.path.join(os.getcwd(), '../..'))
if project_root not in sys.path:
    sys.path.append(project_root)

# Import our backtesting modules
from app.trading.backtester import Backtester
from app.trading.trading_strategies import Strategy, Action
from app.trading.indicators import add_all_indicators
from app.tests.trading.test_helpers import generate_test_data, generate_specific_scenario, validate_backtest_results

## 1. Create Simple Test Strategy

First, let's create a simple strategy for testing purposes. This will allow us to control exactly when trades are executed without relying on market signals.

In [ ]:
class TestStrategy(Strategy):
    """Simple test strategy for testing purposes"""
    
    def __init__(self, name="Test Strategy", initial_balance=10000):
        super().__init__(name=name, initial_balance=initial_balance)
        self.custom_signals = []
    
    def calculate_signals(self, df):
        """Add custom signals to the dataframe"""
        df = df.copy()
        
        # Ensure our custom_signals array matches the dataframe length
        if len(self.custom_signals) < len(df):
            # Pad with None to match length
            self.custom_signals.extend([None] * (len(df) - len(self.custom_signals)))
        elif len(self.custom_signals) > len(df):
            # Trim to match length
            self.custom_signals = self.custom_signals[:len(df)]
        
        # Add signals to dataframe
        df['signal'] = self.custom_signals
        return df
    
    def decide_action(self, current_data):
        """Determine action based on custom signals"""
        if 'signal' not in current_data:
            return Action.HOLD
            
        signal = current_data['signal']
        if signal == 'buy' and not self.position:
            return Action.BUY
        elif signal == 'sell' and self.position:
            return Action.SELL
        else:
            return Action.HOLD

## 2. Generate Test Data

Now we'll generate synthetic data to test our backtester with various market conditions:
- Normal market
- Uptrend
- Downtrend
- Volatile market
- Market crash
- Market rally
- Missing data

In [ ]:
# Generate test data for different scenarios
scenarios = {
    'normal': generate_test_data(days=30, base_price=100, volatility=0.02, trend=0.001),
    'uptrend': generate_specific_scenario('uptrend', days=30, base_price=100),
    'downtrend': generate_specific_scenario('downtrend', days=30, base_price=100),
    'volatile': generate_specific_scenario('volatile', days=30, base_price=100),
    'crash': generate_specific_scenario('crash', days=30, base_price=100),
    'rally': generate_specific_scenario('rally', days=30, base_price=100),
    'missing_data': generate_specific_scenario('missing_data', days=30, base_price=100)
}

# Plot all scenarios to visualize the test data
plt.figure(figsize=(20, 14))
for i, (name, data) in enumerate(scenarios.items(), 1):
    plt.subplot(3, 3, i)
    plt.plot(data['close'])
    plt.title(f'Scenario: {name}')
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 3. Basic Strategy Testing

Let's test a basic strategy with buy and sell signals in different market conditions.

In [ ]:
def run_basic_test(scenario_name):
    """Run a basic test with buy and hold strategy"""
    df = scenarios[scenario_name]
    strategy = TestStrategy(name=f"Test Strategy - {scenario_name}")
    
    # Set up simple buy and hold strategy - buy at day 5, sell at day 25
    signals = [None] * len(df)
    signals[5] = 'buy'
    signals[25] = 'sell'
    strategy.custom_signals = signals
    
    backtester = Backtester(strategy)
    results = backtester.run_backtest(df)
    
    print(f"=== Basic Test Results for {scenario_name} scenario ===")
    print(f"Initial Balance: ${results['initial_balance']}")
    print(f"Final Balance: ${results['final_balance']:.2f}")
    print(f"Total Return: {results['total_return_percent']:.2f}%")
    print(f"Buy & Hold Return: {results['buy_hold_return']:.2f}%")
    
    # Plot the equity curve
    plt.figure(figsize=(10, 6))
    plt.plot(results['portfolio_values'], label='Portfolio Value')
    plt.title(f'Equity Curve for {scenario_name} scenario')
    plt.xlabel('Bar')
    plt.ylabel('Portfolio Value ($)')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.show()
    
    return results

# Test in normal market condition
normal_results = run_basic_test('normal')

# Test in uptrend market condition
uptrend_results = run_basic_test('uptrend')

# Test in downtrend market condition
downtrend_results = run_basic_test('downtrend')

## 4. Testing Risk Management Features

Now let's test the stop-loss and take-profit functionality.

In [ ]:
def test_risk_management(scenario_name, stop_loss_pct=None, take_profit_pct=None):
    """Test stop-loss and take-profit functionality"""
    df = scenarios[scenario_name]
    strategy = TestStrategy(name=f"Risk Management Test - {scenario_name}")
    
    # Set risk management parameters
    if stop_loss_pct is not None:
        strategy.stop_loss_pct = stop_loss_pct
    if take_profit_pct is not None:
        strategy.take_profit_pct = take_profit_pct
    
    # Buy at day 5 but don't sell - let risk management handle it
    signals = [None] * len(df)
    signals[5] = 'buy'
    strategy.custom_signals = signals
    
    backtester = Backtester(strategy)
    results = backtester.run_backtest(df)
    
    print(f"=== Risk Management Test Results for {scenario_name} scenario ===")
    print(f"Stop Loss: {stop_loss_pct*100 if stop_loss_pct is not None else 'None'}%")
    print(f"Take Profit: {take_profit_pct*100 if take_profit_pct is not None else 'None'}%")
    print(f"Initial Balance: ${results['initial_balance']}")
    print(f"Final Balance: ${results['final_balance']:.2f}")
    print(f"Total Return: {results['total_return_percent']:.2f}%")
    
    # Print trade details
    print("\nTrade Details:")
    for trade in results['trades']:
        print(f"- {trade['action']} at ${trade['price']:.2f} on {trade['time']}")
        if 'reason' in trade:
            print(f"  Reason: {trade['reason']}")
    
    return results

# Test stop-loss in volatile scenario
stop_loss_results = test_risk_management('volatile', stop_loss_pct=0.05)

# Test take-profit in uptrend scenario
take_profit_results = test_risk_management('uptrend', take_profit_pct=0.1)

# Test both stop-loss and take-profit in crash scenario
both_results = test_risk_management('crash', stop_loss_pct=0.05, take_profit_pct=0.15)

## 5. Testing Edge Cases

Let's test edge cases such as missing data and extreme price movements.

In [ ]:
def test_edge_case(scenario_name):
    """Test edge case scenarios"""
    df = scenarios[scenario_name]
    strategy = TestStrategy(name=f"Edge Case Test - {scenario_name}")
    
    # Set up a simple strategy
    signals = [None] * len(df)
    signals[5] = 'buy'
    signals[25] = 'sell'
    strategy.custom_signals = signals
    
    backtester = Backtester(strategy)
    results = backtester.run_backtest(df)
    
    print(f"=== Edge Case Test Results for {scenario_name} scenario ===")
    print(f"Initial Balance: ${results['initial_balance']}")
    print(f"Final Balance: ${results['final_balance']:.2f}")
    print(f"Total Return: {results['total_return_percent']:.2f}%")
    
    return results

# Test with missing data
missing_data_results = test_edge_case('missing_data')

# Test with market crash
crash_results = test_edge_case('crash')

# Test with market rally
rally_results = test_edge_case('rally')

## 6. Validating Backtest Results

Now we'll validate the results to ensure our backtester is working correctly.

In [ ]:
def validate_test_results(results, scenario_name):
    """Validate backtest results"""
    is_valid, message = validate_backtest_results(results)
    
    print(f"=== Validation Results for {scenario_name} scenario ===")
    if is_valid:
        print(f"✅ Validation passed: {message}")
    else:
        print(f"❌ Validation failed: {message}")
    
    return is_valid

# Validate results from different tests
validate_test_results(normal_results, 'normal')
validate_test_results(uptrend_results, 'uptrend')
validate_test_results(downtrend_results, 'downtrend')
validate_test_results(stop_loss_results, 'volatile with stop-loss')
validate_test_results(take_profit_results, 'uptrend with take-profit')
validate_test_results(both_results, 'crash with stop-loss and take-profit')
validate_test_results(missing_data_results, 'missing data')

## 7. Creating a Multiple Position Strategy

Let's test a more complex strategy with multiple positions.

In [ ]:
def test_multiple_positions(scenario_name):
    """Test strategy with multiple positions"""
    df = scenarios[scenario_name]
    strategy = TestStrategy(name=f"Multiple Positions Test - {scenario_name}")
    
    # Set up strategy with multiple trades
    signals = [None] * len(df)
    # First trade pair
    signals[3] = 'buy'
    signals[10] = 'sell'
    # Second trade pair
    signals[15] = 'buy'
    signals[22] = 'sell'
    # Third trade pair
    signals[24] = 'buy'
    signals[28] = 'sell'
    
    strategy.custom_signals = signals
    
    backtester = Backtester(strategy)
    results = backtester.run_backtest(df)
    
    print(f"=== Multiple Positions Test Results for {scenario_name} scenario ===")
    print(f"Initial Balance: ${results['initial_balance']}")
    print(f"Final Balance: ${results['final_balance']:.2f}")
    print(f"Total Return: {results['total_return_percent']:.2f}%")
    print(f"Number of trades: {results['total_trades']}")
    print(f"Win rate: {results['win_rate_percent']:.2f}%")
    
    # Plot the equity curve
    plt.figure(figsize=(10, 6))
    plt.plot(results['portfolio_values'], label='Portfolio Value')
    plt.title(f'Equity Curve for {scenario_name} scenario (Multiple Positions)')
    plt.xlabel('Bar')
    plt.ylabel('Portfolio Value ($)')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.show()
    
    return results

# Test in normal market condition
multiple_normal_results = test_multiple_positions('normal')

# Test in volatile market condition
multiple_volatile_results = test_multiple_positions('volatile')

# Validate results
validate_test_results(multiple_normal_results, 'multiple positions - normal')
validate_test_results(multiple_volatile_results, 'multiple positions - volatile')

## 8. Testing Parameter Optimization

Let's test the parameter optimization functionality of our backtester.

In [ ]:
class ParameterizedStrategy(Strategy):
    """Strategy with configurable parameters for optimization testing"""
    
    def __init__(self, name="Parameterized Strategy", initial_balance=10000,
                 ma_short=5, ma_long=20, stop_loss=0.05, take_profit=0.1):
        super().__init__(name=name, initial_balance=initial_balance)
        self.ma_short = ma_short
        self.ma_long = ma_long
        self.stop_loss_pct = stop_loss
        self.take_profit_pct = take_profit
    
    def calculate_signals(self, df):
        """Calculate moving average crossover signals"""
        df = df.copy()
        
        # Calculate moving averages
        df['ma_short'] = df['close'].rolling(window=self.ma_short).mean()
        df['ma_long'] = df['close'].rolling(window=self.ma_long).mean()
        
        # Generate signals - buy when short MA crosses above long MA, sell when it crosses below
        df['signal'] = 0
        df.loc[df['ma_short'] > df['ma_long'], 'signal'] = 1
        df.loc[df['ma_short'] <= df['ma_long'], 'signal'] = -1
        
        # Calculate signal changes
        df['signal_change'] = df['signal'].diff()
        
        return df
    
    def decide_action(self, current_data):
        """Determine action based on signals"""
        if 'signal_change' not in current_data:
            return Action.HOLD
        
        # Buy signal: short MA crosses above long MA
        if current_data['signal_change'] > 0:
            return Action.BUY
        
        # Sell signal: short MA crosses below long MA
        elif current_data['signal_change'] < 0 and self.position:
            return Action.SELL
        
        # Otherwise hold
        return Action.HOLD

def test_parameter_optimization(scenario_name):
    """Test parameter optimization"""
    df = scenarios[scenario_name]
    
    # Define parameter grid for optimization
    param_grid = {
        'ma_short': [3, 5, 8],
        'ma_long': [15, 20, 25],
        'stop_loss': [0.05, 0.1],
        'take_profit': [0.1, 0.2]
    }
    
    # Create strategy and backtester
    strategy = ParameterizedStrategy()
    backtester = Backtester(strategy)
    
    print(f"=== Parameter Optimization Test for {scenario_name} scenario ===")
    print(f"Parameter grid size: {len(param_grid['ma_short']) * len(param_grid['ma_long']) * len(param_grid['stop_loss']) * len(param_grid['take_profit'])} combinations")
    
    # Run optimization
    best_params, best_result = backtester.optimize_strategy_parameters(
        df, param_grid, metric='total_return_percent', maximize=True
    )
    
    print("\nBest Parameters:")
    pprint(best_params)
    
    print("\nBest Result Metrics:")
    metrics = {k: v for k, v in best_result.items() if isinstance(v, (int, float))}
    pprint(metrics)
    
    # Create strategy with best parameters and run again to confirm
    best_strategy = ParameterizedStrategy(**best_params)
    backtester_best = Backtester(best_strategy)
    final_results = backtester_best.run_backtest(df)
    
    print("\nConfirming Best Strategy Results:")
    print(f"Total Return: {final_results['total_return_percent']:.2f}%")
    print(f"Win Rate: {final_results['win_rate_percent']:.2f}%")
    
    # Plot equity curve for best parameters
    plt.figure(figsize=(10, 6))
    plt.plot(final_results['portfolio_values'], label='Portfolio Value')
    plt.title(f'Equity Curve for {scenario_name} scenario (Best Parameters)')
    plt.xlabel('Bar')
    plt.ylabel('Portfolio Value ($)')
    plt.grid(True, alpha=0.3)
    plt.legend()
    plt.show()
    
    return best_params, final_results

# Test in normal market condition
best_params_normal, opt_normal_results = test_parameter_optimization('normal')

# Test in uptrend market condition
best_params_uptrend, opt_uptrend_results = test_parameter_optimization('uptrend')

## 9. Walk-Forward Analysis Testing

Let's test the walk-forward analysis functionality of our backtester.

In [ ]:
def test_walk_forward_analysis(scenario_name):
    """Test walk-forward analysis"""
    # For walk-forward analysis, we need a longer dataset
    df = generate_test_data(days=100, base_price=100, 
                            volatility=0.02, trend=0.001 if scenario_name == 'normal' else 
                                                (0.005 if scenario_name == 'uptrend' else -0.005))
    
    # Define parameter grid for optimization
    param_grid = {
        'ma_short': [3, 5, 8],
        'ma_long': [15, 20, 25]
    }
    
    # Create strategy and backtester
    strategy = ParameterizedStrategy()
    backtester = Backtester(strategy)
    
    print(f"=== Walk-Forward Analysis Test for {scenario_name} scenario ===")
    
    # Run walk-forward analysis
    wfa_results = backtester.walk_forward_analysis(
        df, param_grid, 
        window_size=30,  # 30-day windows
        step_size=10,    # 10-day steps
        metric='total_return_percent', 
        maximize=True
    )
    
    print(f"Number of windows analyzed: {len(wfa_results)}")
    
    # Display parameter stability across windows
    for i, (params, results) in enumerate(wfa_results):
        print(f"\nWindow {i+1} Best Parameters:")
        pprint(params)
        print(f"Window {i+1} Performance: {results['total_return_percent']:.2f}%")
    
    # Calculate parameter stability metrics
    param_values = {}
    for params, _ in wfa_results:
        for param, value in params.items():
            if param not in param_values:
                param_values[param] = []
            param_values[param].append(value)
    
    print("\nParameter Stability Analysis:")
    for param, values in param_values.items():
        mean_val = np.mean(values)
        std_val = np.std(values)
        cv = std_val / mean_val if mean_val != 0 else 0  # Coefficient of variation
        
        print(f"{param}: mean={mean_val:.2f}, std={std_val:.2f}, CV={cv:.2f}")
        print(f"Values: {values}")
    
    # Plot equity curves for all windows
    plt.figure(figsize=(12, 6))
    
    for i, (params, result) in enumerate(wfa_results):
        # Each window has its own equity curve
        plt.plot(result['portfolio_values'], label=f"Window {i+1}")
    
    plt.title(f'Walk-Forward Analysis Equity Curves ({scenario_name} scenario)')
    plt.xlabel('Bar within Window')
    plt.ylabel('Portfolio Value ($)')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.show()
    
    return wfa_results

# Test in normal market condition
wfa_normal_results = test_walk_forward_analysis('normal')

# Test in uptrend market condition
wfa_uptrend_results = test_walk_forward_analysis('uptrend')

## 10. Running Automated Tests

Finally, let's demonstrate how to run automated unit tests for the backtester.

In [ ]:
def run_unit_tests():
    """Run backtester unit tests"""
    # Import the unit tests
    from app.tests.trading.test_backtester import BacktesterTests
    
    # Create a test suite with all tests from BacktesterTests
    suite = unittest.TestLoader().loadTestsFromTestCase(BacktesterTests)
    
    # Run the tests and capture the results
    result = unittest.TextTestRunner(verbosity=2).run(suite)
    
    # Print summary
    print("\nTest Summary:")
    print(f"Ran {result.testsRun} tests")
    print(f"Failures: {len(result.failures)}")
    print(f"Errors: {len(result.errors)}")
    print(f"Skipped: {len(result.skipped)}")
    
    # Return True if all tests passed
    return len(result.failures) == 0 and len(result.errors) == 0

# Run the unit tests
all_tests_passed = run_unit_tests()
print(f"\nAll tests passed: {all_tests_passed}")

## 11. Conclusion

In this notebook, we've demonstrated how to:

1. Create systematic tests for our backtesting framework
2. Test various market scenarios and edge cases
3. Validate the correctness of backtesting results
4. Test advanced features like parameter optimization and walk-forward analysis
5. Run automated unit tests

The testing framework provides confidence that our backtester works as expected and handles various market conditions appropriately.